# Prediction in Arbitrary Resolution for fODF INR

Code by:

Tyler Spears - tas6hh@virginia.edu

Dr. Tom Fletcher

## Imports & Setup

In [1]:
# Imports
# Automatically re-import project-specific modules.
%load_ext autoreload
%autoreload 2

# imports
import collections
import copy
import datetime
import functools
import inspect
import io
import itertools
import math
import os
import pathlib
import pdb
import random
import shutil
import subprocess
import sys
import tempfile
import time
import typing
import warnings
import zipfile
from functools import partial
from pathlib import Path
from pprint import pprint as ppr

import aim
import dotenv
import einops

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import monai

# Data management libraries.
import nibabel as nib
import nibabel.processing

# Computation & ML libraries.
import numpy as np
import pandas as pd
import seaborn as sns
import skimage
import torch
import torch.nn.functional as F
from box import Box
from icecream import ic
from natsort import natsorted

import pitn

from inr_networks import INREncoder, Decoder

plt.rcParams.update({"figure.autolayout": True})
plt.rcParams.update({"figure.facecolor": [1.0, 1.0, 1.0, 1.0]})
plt.rcParams.update({"image.cmap": "gray"})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True, threshold=100, linewidth=88)
torch.set_printoptions(sci_mode=False, threshold=100, linewidth=88)

In [2]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = "direnv exec {} /usr/bin/env".format(os.getcwd())
# Run command in a new subprocess.
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode("utf-8")
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True)

direnv: loading ~/Projects/pitn/.envrc


True

In [3]:
# torch setup
# allow for CUDA usage, if available
if torch.cuda.is_available():
    # Pick only one device for the default, may use multiple GPUs for training later.
    if "CUDA_PYTORCH_DEVICE_IDX" in os.environ.keys():
        dev_idx = int(os.environ["CUDA_PYTORCH_DEVICE_IDX"])
    else:
        dev_idx = 1
    device = torch.device(f"cuda:{dev_idx}")
    print("CUDA Device IDX ", dev_idx)
    torch.cuda.set_device(device)
    print("CUDA Current Device ", torch.cuda.current_device())
    print("CUDA Device properties: ", torch.cuda.get_device_properties(device))
    # The flag below controls whether to allow TF32 on matmul. This flag defaults to False
    # in PyTorch 1.12 and later.
    torch.backends.cuda.matmul.allow_tf32 = True
    # See
    # <https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices>
    # for details.

    # Activate cudnn benchmarking to optimize convolution algorithm speed.
    if torch.backends.cudnn.enabled:
        torch.backends.cudnn.benchmark = True
        print("CuDNN convolution optimization enabled.")
        # The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
        torch.backends.cudnn.allow_tf32 = True

else:
    device = torch.device("cpu")
# keep device as the cpu
# device = torch.device('cpu')
print(device)

CUDA Device IDX  1
CUDA Current Device  1
CUDA Device properties:  _CudaDeviceProperties(name='NVIDIA RTX A5000', major=8, minor=6, total_memory=24245MB, multi_processor_count=64)
CuDNN convolution optimization enabled.
cuda:1


In [4]:
%%capture --no-stderr cap
# Capture output and save to log. Needs to be at the *very first* line of the cell.
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
if torch.cuda.is_available():
    # GPU information
    try:
        gpu_info = pitn.utils.system.get_gpu_specs()
        print(gpu_info)
    except NameError:
        print("CUDA Version: ", torch.version.cuda)
else:
    print("CUDA not in use, falling back to CPU")

In [5]:
# cap is defined in an ipython magic command
try:
    print(cap)
except NameError:
    pass

Author: Tyler Spears

Last updated: 2023-06-19T23:16:59.107177-04:00

Python implementation: CPython
Python version       : 3.10.9
IPython version      : 8.4.0

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.19.0-41-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

Git hash: aaed9ec4776e4f352c5569a9295bfa47cb662d9d

pitn      : 0.0.post1.dev291+ge9e5f58.d20230531
einops    : 0.6.0
monai     : 1.2.0
torch     : 1.13.1
nibabel   : 5.1.0
sys       : 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:20:04) [GCC 11.3.0]
seaborn   : 0.12.1
numpy     : 1.23.4
matplotlib: 3.5.2
skimage   : 0.19.3
aim       : 3.14.4
pandas    : 1.5.2

==================================================GPU Specs==================================================
  id  Name              Driver Version      CUDA Version  Total Memory    uuid
----  ----------------  ----------------  --------------  --------------  ----------------------------------------
  

## Experiment & Parameters Setup

In [6]:
p = Box(default_box=True)
# Experiment defaults, can be overridden in a config file.

# General experiment-wide params
###############################################
p.target_vox_spacing = 1.0
p.results_dir = "/data/srv/outputs/pitn/results/runs"
p.tmp_results_dir = "/data/srv/outputs/pitn/results/tmp"
# p.train_val_test_split_file = random.choice(
#     list(Path("./data_splits").glob("HCP*train-val-test_split*.csv"))
# )
p.model_weight_f = str(
    Path(p.tmp_results_dir)
    / "2023-06-18T20_34_58"
    / "state_dict_epoch_99_step_25000.pt"
)
###############################################
# kwargs for the sub-selection function to go from full DWI -> low-res DWI.
# See `sub_select_dwi_from_bval` function in `pitn`.
p.bval_sub_sample_fn_kwargs = dict(
    shells_to_remove=[2000],
    within_shell_idx_to_keep={
        0: range(0, 9),
        1000: range(0, 45),
        3000: range(0, 45),
    },
)
# 1.25mm -> 2.0mm
p.baseline_lr_spacing_scale = 1.6
p.scale_prefilter_kwargs = dict(
    sigma_scale_coeff=2.5,
    sigma_truncate=4.0,
)
p.test.subj_ids = list(
    map(
        str,
        [
            581450,
        ],
    )
)

# Network/model parameters.
p.encoder = dict(
    interior_channels=80,
    out_channels=128,
    n_res_units=3,
    n_dense_units=3,
    activate_fn="relu",
    input_coord_channels=True,
)
p.decoder = dict(
    context_v_features=128,
    in_features=p.encoder.out_channels,
    out_features=45,
    m_encode_num_freqs=36,
    sigma_encode_scale=3.0,
)


# If a config file exists, override the defaults with those values.
try:
    if "PITN_CONFIG" in os.environ.keys():
        config_fname = Path(os.environ["PITN_CONFIG"])
    else:
        config_fname = pitn.utils.system.get_file_glob_unique(Path("."), r"config.*")
    f_type = config_fname.suffix.casefold()
    if f_type in {".yaml", ".yml"}:
        f_params = Box.from_yaml(filename=config_fname)
    elif f_type == ".json":
        f_params = Box.from_json(filename=config_fname)
    elif f_type == ".toml":
        f_params = Box.from_toml(filename=config_fname)
    else:
        raise RuntimeError()

    p.merge_update(f_params)

except:
    print("WARNING: Config file not loaded")
    pass

# Remove the default_box behavior now that params have been fully read in.
_p = Box(default_box=False)
_p.merge_update(p)
p = _p

In [7]:
# tvt_split = pd.read_csv(p.train_val_test_split_file)
# p.test.subj_ids = natsorted(tvt_split[tvt_split.split == "test"].subj_id.tolist())

In [8]:
ic(p.to_dict())

ic| p.to_dict(): {'baseline_lr_spacing_scale': 1.6,
                  'bval_sub_sample_fn_kwargs': {'shells_to_remove': [2000],
                                                'within_shell_idx_to_keep': {0: range(0, 9),
                                                                             1000: range(0, 45),
                                                                             3000: range(0, 45)}},
                  'decoder': {'context_v_features': 128,
                              'in_features': 128,
                              'm_encode_num_freqs': 36,
                              'out_features': 45,
                              'sigma_encode_scale': 3.0},
                  'encoder': {'activate_fn': 'relu',
                              'input_coord_channels': True,
                              'interior_channels': 80,
                              'n_dense_units': 3,
                              'n_res_units': 3,
                              'out_channels': 

{'target_vox_spacing': 1.0,
 'results_dir': '/data/srv/outputs/pitn/results/runs',
 'tmp_results_dir': '/data/srv/outputs/pitn/results/tmp',
 'model_weight_f': '/data/srv/outputs/pitn/results/tmp/2023-06-18T20_34_58/state_dict_epoch_99_step_25000.pt',
 'bval_sub_sample_fn_kwargs': {'shells_to_remove': [2000],
  'within_shell_idx_to_keep': {0: range(0, 9),
   1000: range(0, 45),
   3000: range(0, 45)}},
 'baseline_lr_spacing_scale': 1.6,
 'scale_prefilter_kwargs': {'sigma_scale_coeff': 2.5, 'sigma_truncate': 4.0},
 'test': {'subj_ids': ['581450']},
 'encoder': {'interior_channels': 80,
  'out_channels': 128,
  'n_res_units': 3,
  'n_dense_units': 3,
  'activate_fn': 'relu',
  'input_coord_channels': True},
 'decoder': {'context_v_features': 128,
  'in_features': 128,
  'out_features': 45,
  'm_encode_num_freqs': 36,
  'sigma_encode_scale': 3.0}}

## Data Loading

In [9]:
hcp_full_res_data_dir = Path("/data/srv/data/pitn/hcp")
hcp_full_res_fodf_dir = Path("/data/srv/outputs/pitn/hcp/full-res/fodf")

assert hcp_full_res_data_dir.exists()
assert hcp_full_res_fodf_dir.exists()

In [10]:
# Define hte bval/bvec sub-sample scheme according to the parameter dict kwargs.
bval_sub_sample_fn = partial(
    pitn.data.datasets2.sub_select_dwi_from_bval,
    **p.bval_sub_sample_fn_kwargs.to_dict(),
)

### Test Dataset

In [11]:
with warnings.catch_warnings(record=True) as warn_list:

    test_dataset = pitn.data.datasets2.HCPfODFINRDataset(
        subj_ids=p.test.subj_ids,
        dwi_root_dir=hcp_full_res_data_dir,
        fodf_root_dir=hcp_full_res_fodf_dir,
        transform=pitn.data.datasets2.HCPfODFINRDataset.default_pre_sample_tf(
            sample_mask_key="wm_mask",
            bval_sub_sample_fn=bval_sub_sample_fn,
        ),
    )

    test_dataset = pitn.data.datasets2.HCPfODFINRWholeBrainDataset(
        test_dataset,
        transform=pitn.data.datasets2.HCPfODFINRWholeBrainDataset.default_vol_tf(
            baseline_iso_scale_factor_lr_spacing_mm_low_high=p.baseline_lr_spacing_scale,
            scale_prefilter_kwargs=p.scale_prefilter_kwargs,
        ),
    )
    test_dataset = monai.data.CacheDataset(
        test_dataset,
        cache_num=1,
        transform=None,
        progress=True,
        copy_cache=False,
        num_workers=1,
    )

print("=" * 10)
print("Warnings caught:")
ws = "\n".join(
    [
        warnings.formatwarning(
            w.message, w.category, w.filename, w.lineno, w.file, w.line
        )
        for w in warn_list
    ]
)
ws = "\n".join(filter(lambda s: bool(s.strip()), ws.splitlines()))
print(ws, flush=True)
print("=" * 10)

Loading dataset: 100%|██████████| 1/1 [01:08<00:00, 68.60s/it]

Warnings caught:



## Testing

In [12]:
def batchwise_masked_mse(y_pred, y, mask):
    masked_y_pred = y_pred.clone()
    masked_y = y.clone()
    masked_y_pred[~mask] = torch.nan
    masked_y[~mask] = torch.nan
    se = F.mse_loss(masked_y_pred, masked_y, reduction="none")
    se = se.reshape(se.shape[0], -1)
    mse = torch.nanmean(se, dim=1)
    return mse

In [13]:
ts = datetime.datetime.now().replace(microsecond=0).isoformat()
# Break ISO format because many programs don't like having colons ':' in a filename.
ts = ts.replace(":", "_")
experiment_name = f"{ts}_inr-pred-test_super-res_non-init-decoder_blank-pred"
tmp_res_dir = Path(p.tmp_results_dir) / experiment_name
tmp_res_dir.mkdir(parents=True)

In [24]:
ic(sum(p.numel() for p in encoder.parameters()))
ic(sum(p.numel() for p in decoder.parameters()))
ic(128 * 128 + 6 * (350 * 128) + 128 * 45)
ic(128 * 256 + 3 * (256 * 256) + 256 * 45)
ic(128 * 128 + 6 * (350 * 128) + 128 * 45)

ic| sum(p.numel() for p in encoder.parameters()): 4667306
ic| sum(p.numel() for p in decoder.parameters()): 291885
ic| 128 * 128 + 6*(350 * 128 ) + 128 * 45: 290944
ic| 128 * 256 + 3*(256*256) + 256*45: 240896
ic| 128 * 128 + 6*(350 * 128 ) + 128 * 45: 290944


290944

In [34]:
test_dataset[0]["lr_dwi"].shape

torch.Size([99, 77, 100, 77])

In [40]:
e = INREncoder(
    in_channels=99 + 3,
    n_res_units=3,
    n_dense_units=3,
    activate_fn="relu",
    input_coord_channels=True,
    interior_channels=72,
    out_channels=96,
)
e(torch.randn(1, 102, 36, 36, 36, dtype=torch.float32))
e_p = ic(sum(p.numel() for p in e.parameters()))
d = Decoder(
    context_v_features=96,
    in_features=96,
    out_features=45,
    m_encode_num_freqs=20,
    sigma_encode_scale=3.0,
)
d_p = ic(sum(p.numel() for p in d.parameters()))
ic(e_p + d_p)
r = INREncoder(
    interior_channels=48,
    out_channels=9,
    n_res_units=2,
    n_dense_units=2,
    in_channels=96,
    activate_fn="relu",
    input_coord_channels=False,
)
r(torch.randn(1, 96, 36, 36, 36, dtype=torch.float32))
r_p = ic(sum(p.numel() for p in r.parameters()))

ic| sum(p.numel() for p in e.parameters()): 3761178
ic| sum(p.numel() for p in d.parameters()): 142125
ic| e_p + d_p: 3903303
ic| sum(p.numel() for p in r.parameters()): 966483


In [25]:
model = "INR"
model_pred_res_dir = tmp_res_dir / model
# model_pred_res_dir.mkdir(exist_ok=True)
# with open(model_pred_res_dir / "model_description.txt", "x") as f:
#     f.write(f"model weights file: {str(p.model_weight_f)}\n")
#     f.write(f"encoder parameters: \n{str(p.encoder.to_dict())}\n")
#     f.write(f"decoder parameters: \n{str(p.decoder.to_dict())}\n")

# Wrap the entire loop in a try...except statement to save out a failure indicator file.
try:
    system_state_dict = torch.load(p.model_weight_f)
    encoder_state_dict = system_state_dict["encoder"]

    decoder_state_dict = system_state_dict["decoder"]

    if "in_channels" not in p.encoder:
        in_channels = int(test_dataset[0]["lr_dwi"].shape[0]) + 3
    else:
        in_channels = p.encoder.in_channels

    encoder = INREncoder(**{**p.encoder.to_dict(), **{"in_channels": in_channels}})
    # encoder.load_state_dict(encoder_state_dict) #!DEBUG
    encoder.to(device)

    decoder = Decoder(**p.decoder.to_dict())
    # decoder.load_state_dict(decoder_state_dict) #!DEBUG
    decoder.to(device)
    del (
        system_state_dict,
        encoder_state_dict,
        decoder_state_dict,
    )
    # with open(model_pred_res_dir / "model_description.txt", "a") as f:
    #     f.write(f"encoder layers: \n{str(encoder)}\n")
    #     f.write(f"decoder layers: \n{str(decoder)}\n")

    test_dataloader = monai.data.DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=True,
        pin_memory=True,
        num_workers=2,
        persistent_workers=True,
        prefetch_factor=1,
    )

    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        print("Starting inference", flush=True)
        for batch_dict in test_dataloader:
            subj_id = batch_dict["subj_id"]
            if len(subj_id) == 1:
                subj_id = subj_id[0]
            print(f"Starting {subj_id}", flush=True)

            x = batch_dict["lr_dwi"].to(device)
            batch_size = x.shape[0]
            x_mask = batch_dict["lr_brain_mask"].to(torch.bool).to(device)
            x_affine_vox2world = batch_dict["affine_lr_vox2world"].to(device)
            x_vox_size = batch_dict["lr_vox_size"].to(device)
            x_coords = pitn.affine.affine_coordinate_grid(
                x_affine_vox2world, tuple(x.shape[2:])
            )
            # Fix an edge case in the affine_coordinate_grid function.
            if batch_size == 1:
                if x_coords.shape[0] != 1:
                    x_coords.unsqueeze_(0)

            # Calculate the new coordinates given the target vox spacing.
            input_vox_size = x_vox_size.flatten().cpu().numpy()[0]
            scale_x2sr = p.target_vox_spacing / input_vox_size
            # We don't need a particular voxel buffer here, only that the SR fov is
            # totally contained within the src fov by some amount > 0.
            sr_affine_vox2world = pitn.data.datasets2._random_iso_center_scale_affine(
                x_affine_vox2world[0].cpu(),
                x[0].cpu(),
                scale_low=scale_x2sr,
                scale_high=scale_x2sr,
                n_delta_buffer_scaled_vox=0,
            )
            sr_affine_vox2world = sr_affine_vox2world[None]
            sr_spatial_shape = pitn.affine.transform_coords(
                x_coords[0, -1, -1, -1].to(sr_affine_vox2world),
                torch.linalg.inv(sr_affine_vox2world),
            )
            sr_spatial_shape = tuple(
                torch.floor(sr_spatial_shape).int().cpu().numpy().tolist()
            )
            # Be careful to keep all the "super-sized" tensors on the cpu!
            sr_coords = pitn.affine.affine_coordinate_grid(
                sr_affine_vox2world.to(torch.float32).cpu(), sr_spatial_shape
            )

            # Fix an edge case in the affine_coordinate_grid function.
            if batch_size == 1:
                if sr_coords.shape[0] != 1:
                    sr_coords.unsqueeze_(0)

            # Interpolate a mask for the sr volume.
            sr_mask = pitn.affine.sample_vol(
                x_mask.cpu(),
                sr_coords.cpu(),
                affine_vox2mm=x_affine_vox2world.cpu(),
                mode="nearest",
                align_corners=True,
            )

            # Concatenate the input world coordinates as input features into the
            # encoder. Mask out the x coordinates that are not to be considered.
            x_coord_mask = einops.rearrange(x_mask, "b 1 x y z -> b x y z 1")
            x_coords_encoder = einops.rearrange(
                x_coords * x_coord_mask, "b x y z coord -> b coord x y z"
            )
            x = torch.cat([x, x_coords_encoder], dim=1)
            ctx_v = encoder(x)
            #!DEBUG>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
            # meds = torch.median(
            #     ctx_v.reshape(ctx_v.shape[1], -1), dim=1
            # ).values.reshape(1, ctx_v.shape[1], 1, 1, 1)
            ctx_v = ctx_v * 0  # + meds

            # sr_coords = sr_coords * 0
            #!DEBUG<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

            # Whole-volume inference is memory-prohibitive, so use a sliding
            # window inference method on the encoded volume.
            # Transform y_coords into a coordinates-first shape, for the interface, and
            # attach the mask for compatibility with the sliding inference function.
            sr_slide_window = torch.cat(
                [
                    einops.rearrange(sr_coords.cpu(), "b x y z coord -> b coord x y z"),
                    sr_mask.to(sr_coords).cpu(),
                ],
                dim=1,
            )
            fn_coordify = lambda x: einops.rearrange(
                x, "b coord x y z -> b x y z coord"
            )
            # Keep the whole volume on the CPU, and only transfer the sliding windows
            # to the GPU.
            pred_fodf = monai.inferers.sliding_window_inference(
                sr_slide_window.cpu(),
                roi_size=(96, 96, 96),
                sw_batch_size=batch_size,
                predictor=lambda q: decoder(
                    # Rearrange back into coord-last format.
                    query_world_coord=fn_coordify(q[:, :-1]).to(
                        device, non_blocking=True
                    ),
                    query_world_coord_mask=fn_coordify(q[:, -1:].bool()).to(
                        device, non_blocking=True
                    ),
                    context_v=ctx_v,
                    context_world_coord_grid=x_coords,
                    affine_context_vox2world=x_affine_vox2world,
                    affine_query_vox2world=sr_affine_vox2world,
                    context_vox_size_world=x_vox_size,
                    query_vox_size_world=torch.ones_like(x_vox_size)
                    * p.target_vox_spacing,
                ).cpu(),
                overlap=0,
                padding_mode="replicate",
            ).cpu()
            print(f"Done with inference subject {subj_id}", flush=True)
            # Mask out the prediction, otherwise the file size will be considerably
            # larger.
            pred_fodf *= sr_mask.cpu()
            # Write out prediction to a .nii.gz file.
            pred_f = (
                model_pred_res_dir
                / f"{subj_id}_{model}_prediction_{input_vox_size}mm-to-{p.target_vox_spacing}mm.nii.gz"
            )
            pred_affine = sr_affine_vox2world[0].cpu().numpy()
            pred_fodf_vol = einops.rearrange(
                pred_fodf.detach().cpu().numpy(), "1 c x y z -> x y z c"
            ).astype(np.float32)
            # pred_im = nib.Nifti1Image(pred_fodf_vol, affine=pred_affine)

            # print(f"Saving prediction {subj_id}", flush=True)
            # nib.save(pred_im, pred_f)
            fig = plt.figure(dpi=150)
            pitn.viz.plot_fodf_coeff_slices(
                pred_fodf[0], interpolation="nearest", fig=fig
            )
            print(f"Finished {subj_id}", flush=True)

except KeyboardInterrupt as e:
    (tmp_res_dir / "STOPPED").touch()
    raise e
except Exception as e:
    (tmp_res_dir / "FAILED").touch()
    raise e

/opt/mambaforge/envs/pitn/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning:

Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.



Starting inference
Starting 581450


KeyboardInterrupt: 